# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key
from api_keys import geo_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = 'output_data/cities.csv'
city_weather = pd.read_csv(path, encoding='utf-8')
# city_test = pd.DataFrame('/output_data/cities.csv')
city_weather.head()



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_weather[["Lat", "Lng"]].astype(float)
humidity = city_weather['Humidity'].astype(float)


In [5]:
# Create a weather Heatmap layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal = city_weather.loc[((city_weather['Humidity'] < 70) &
                          (city_weather['Humidity'] > 50) &
                          (city_weather['Max Temp'] > 75.0) & 
                          (city_weather['Max Temp'] < 85.0) & 
                          (city_weather['Cloudiness'] < 30))]
# ideal.count()
ideal =ideal.dropna()
ideal.count()

City_ID       9
City          9
Cloudiness    9
Country       9
Date          9
Humidity      9
Lat           9
Lng           9
Max Temp      9
Wind Speed    9
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = ideal
hotel_df['Hotel Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
153,153,porto novo,20,BR,1558378783,53,-23.68,-45.44,77.00,4.70,
237,237,andradas,20,BR,1558378535,56,-22.07,-46.57,79.00,6.93,
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,
439,439,vila do maio,20,CV,1558378879,54,15.13,-23.22,80.60,13.87,
457,457,lahaina,20,US,1558378883,61,20.87,-156.68,82.40,11.41,
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,
527,527,tomatlan,0,MX,1558378894,59,19.93,-105.25,84.66,7.90,


In [24]:
apiKey = geo_key
hotel_location = hotel_df.loc[(hotel_df['City'] == 'andradas')][['Lng', 'Lat']]
hotel_location
# categories = 'accommodation.hotel'
# locate = f'filter=circle:{hotel_location},10000'
# base_url = f'https://api.geoapify.com/v2/places?filter=circle:-46.57,-22.07,10000&categories={categories}&apiKey={apiKey}'
# hotel_response = requests.get(base_url).json()
# print(json.dumps(hotel_response, indent=4, sort_keys=True))



# use iterrows to iterate through pandas dataframe
# for index, row in types_df.iterrows():

#     # get restaurant type from df
#     restr_type = row['ethnicity']

#     # add keyword to params dict
#     params['keyword'] = restr_type

#     # assemble url and make API request
#     print(f"Retrieving Results for Index {index}: {restr_type}.")
#     response = requests.get(base_url, params=params).json()
    
#     # extract results
#     results = response['results']
    
#     try:
#         print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

,Lng,Lat
237,-46.57,-22.07


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
